In [1]:
import pandas as pd
import numpy as np
import json

#cargo los datasets
#si agrego  dtype='unicode' evito algunos errores sobre tipos de datos
dfMovies = pd.read_csv(r'datasets\movies_dataset.csv', dtype='unicode')
dfCredits= pd.read_csv(r'datasets\credits.csv', dtype='unicode')

pd.set_option("display.max_rows", 2000, "display.max_columns", None)

In [2]:
#TRANSFORMACIONES VARIAS ---> NO OLVIDAR CORRER LO QUE ESTA COMENTADO

#lleno NaNs con 0s en las columnas revenue y budget
#elimino rows que contienen un NaN en la columna release_date
#elimino columnas que no se utilizaran
try:
    dfMovies['revenue'] = dfMovies['revenue'].fillna(0)
    dfMovies['budget'] = dfMovies['budget'].fillna(0)
    dfMovies = dfMovies.dropna(subset='release_date')
    dfMovies= dfMovies.drop(columns=["video","adult","imdb_id" ,"original_title", "poster_path", "homepage"])
except:
    print("estas transformaciones ya se ejecutaron")


In [3]:
#FUNCIONES

#desanida columna y devuelve un dataframe
def desanidarColumna(columna):
    lista = []
    listaDict=[]
    counter=0
    for k, v in columna.items():
        if isinstance(v, str):
            lista.append(v)
        else:
            lista.append(None)
    for i in lista:   
                try:
                    i=json.loads(lista[counter].replace("'", '"'))
                    listaDict.append(i)
                    counter+=1
                except:
                    listaDict.append(None)
                    counter+=1
    listaDict=pd.json_normalize(listaDict)
    return listaDict

def desanidarVariasColumnas(columna):
    lista=[]
    for col in columna:            
        #normalizo cada columna
        colAnidada=pd.json_normalize(columna[col])
        lista.append(colAnidada) #meto cada columna en una lista

        #armo un df concatenando cada columna de la lista
        nuevoDF=pd.concat(lista)
    return nuevoDF


In [4]:
#desanido belongs to collection

Títulos = (desanidarColumna(dfMovies['belongs_to_collection']))
#dfMovies["Títulos"] = Títulos["name"] es otra forma de agregar la columna
#agrego la columna al inicio

#inserto nueva columna y elimino la vieja
dfMovies.insert(0, "movie_title", Títulos["name"])
dfMovies= dfMovies.drop(columns=["belongs_to_collection"])

In [5]:
#la columna generos hay que desanidarle cada columna despues de aplciarle la funcion

dfGeneros = (desanidarColumna(dfMovies['genres'])) #le aplico la primera funcion
dfGeneros=desanidarVariasColumnas(dfGeneros) #desanido cada columna con la segunda funcion      
dfGeneros=dfGeneros.iloc[:45378] #esto es cualquier cosa pero bueno, funciona xd
                                 #uso iloc para obtener solo hasta el row 45378 porque sino tengo errores
                                 
#inserto nueva columna y elimino la vieja
dfMovies.insert(1, "movie_genre", dfGeneros["name"])
dfMovies= dfMovies.drop(columns=["genres"])

In [6]:
#desanido tabla production_companies
dfCompanies=desanidarColumna(dfMovies['production_companies'])
dfCompanies=desanidarVariasColumnas(dfCompanies)
dfCompanies= dfCompanies.iloc[:45379]

#inserto nueva columna y elimino la vieja
dfMovies.insert(7, "production_company", dfCompanies["name"])
dfMovies= dfMovies.drop(columns=["production_companies"])

In [7]:
#desanido production_countries
dfProdCountries=desanidarVariasColumnas(desanidarColumna(dfMovies["production_countries"]))
dfProdCountries = dfProdCountries.iloc[:45379]

#inserto nueva columna y elimino la vieja
dfMovies.insert(8, "production_country", dfProdCountries["name"])
dfMovies= dfMovies.drop(columns=["production_countries"])

In [8]:
#desanido spoken_languages
dfSpokenLan=desanidarVariasColumnas(desanidarColumna(dfMovies["spoken_languages"]))
dfSpokenLan = dfSpokenLan.iloc[:45379]

#inserto nueva columna y elimino la vieja
dfMovies.insert(12, "language", dfSpokenLan["name"]) #puedo elegir entre poner la abreviacion o la palabra entera
dfMovies= dfMovies.drop(columns=["spoken_languages"])

In [9]:
#tablas que contienen fechas:
    #release_date
release_year = dfMovies["release_date"]
#cambio valores de fechas invalidas a fechas default
release_year[19730] = "1900-01-01"
release_year[29503] = "1900-01-01"
release_year[35587] = "1900-01-01"

#obtengo solo año de cada string de fecha y lo meto en release_year
release_year = pd.to_datetime(release_year).dt.year

#inserto nueva columan release_year
dfMovies.insert(10, "release_year", release_year)

#se supone que transforme la columna en date type pero es un int REVISAR

In [99]:
budget=dfMovies["budget"]
budget[19730] = 0
budget[29503] = 0
budget[35587] = 0

revenue=pd.DataFrame(dfMovies["revenue"]).astype(float)
budget=pd.DataFrame(dfMovies["budget"]).astype(float)


dfMovies["return"] = np.where(revenue > 0,revenue/budget, np.nan)
dfMovies

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10260\2460657911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget[19730] = 0
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10260\2460657911.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget[29503] = 0
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10260\2460657911.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget[35587] = 0


ValueError: Expected a 1D array, got an array with shape (45379, 2)